<a href="https://colab.research.google.com/github/Ramyaanbazhagan/Ramyaanbazhagan/blob/main/Training%20a%20Hebbian%20Neural%20Network%20architecture%20with%20a%20finance%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import spacy
import sqlite3
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load NLP model
nlp = spacy.load("en_core_web_sm")

def fetch_wikipedia_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all("p")
        return "\n".join([para.get_text() for para in paragraphs])
    return ""

def preprocess_text(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

class SimpleChatbot:
    def __init__(self, db_name="chatbot.db"):
        self.db_name = db_name
        self.create_database()
        self.load_data()

    def create_database(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS knowledge (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                sentence TEXT
            )
        """)
        conn.commit()
        conn.close()

    def store_data(self, text):
        sentences = [sent.text.strip() for sent in nlp(text).sents]
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        cursor.executemany("INSERT INTO knowledge (sentence) VALUES (?)", [(s,) for s in sentences])
        conn.commit()
        conn.close()

    def load_data(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        cursor.execute("SELECT sentence FROM knowledge")
        self.sentences = [row[0] for row in cursor.fetchall()]
        conn.close()

        if self.sentences:
            self.vectorizer = TfidfVectorizer()
            self.tfidf_matrix = self.vectorizer.fit_transform(self.sentences)
        else:
            self.vectorizer = None
            self.tfidf_matrix = None

    def get_response(self, query):
        if not self.sentences:
            return "I don't have any information stored yet."

        query_vec = self.vectorizer.transform([query])
        similarity = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        best_match_index = similarity.argmax()

        if similarity[best_match_index] > 0.2:  # Increase threshold to filter irrelevant responses
            return self.sentences[best_match_index].split('.')[0] + "."  # Return only the first sentence
        else:
            return "I'm not sure about that."

# Fetch Wikipedia content
url = "https://en.wikipedia.org/wiki/Animal"
document_text = fetch_wikipedia_content(url)

# Initialize chatbot
chatbot = SimpleChatbot()
chatbot.store_data(document_text)
chatbot.load_data()

# User interaction
while True:
    user_input = input("Ask a question (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    response = chatbot.get_response(user_input)
    print("Chatbot:", response)

OperationalError: no such column: sentence

In [12]:
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

# Step 1: Define activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Step 2: Define loss function
def loss_function(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Step 3: Implement Hebbian Neural Network class
class HebbianNN:
    def __init__(self, input_size, output_size, learning_rate=0.01):
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.learning_rate = learning_rate

    def forward(self, X):
        self.input = X
        self.output = sigmoid(np.dot(X, self.weights))
        return self.output

    def hebbian_update(self):
        self.weights += self.learning_rate * np.dot(self.input.T, self.output)

    def train(self, X, y, epochs=100):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = loss_function(y, y_pred)
            self.hebbian_update()
            print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

# Step 4: Load and preprocess the historical stock prices of Apple (AAPL)
data = yf.download('AAPL', start='2010-01-01', end='2022-02-26')
X = data[['Open', 'High', 'Low', 'Volume']]
y = data['Close'].values.reshape(-1, 1)

# Scale the data using Min-Max Scaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

# Step 5: Train the Hebbian Neural Network
nn = HebbianNN(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1], learning_rate=0.01)
nn.train(X_scaled, y_scaled, epochs=10)

[*********************100%***********************]  1 of 1 completed

Epoch 1, Loss: 0.1387
Epoch 2, Loss: 0.3866
Epoch 3, Loss: 0.5950
Epoch 4, Loss: 0.6561
Epoch 5, Loss: 0.6729
Epoch 6, Loss: 0.6780
Epoch 7, Loss: 0.6796
Epoch 8, Loss: 0.6802
Epoch 9, Loss: 0.6804
Epoch 10, Loss: 0.6805
